# Research Questions
"""
This notebook addresses the following research questions:

3a. What is the best configuration for training a waste classification model using CNNs?
3b. How do different CNN architectures, layer configurations, and filter sizes impact classification accuracy based on the image dataset used?

To answer these questions, we will:
1. Implement and compare different CNN architectures (Simple CNN, VGG-style, ResNet-style, MobileNet-style)
2. Test various filter sizes (3x3, 5x5, 7x7)
3. Experiment with different network depths (2-5 convolutional blocks)
4. Evaluate regularization techniques (Dropout, L2, Batch Normalization)
5. Analyze how each configuration impacts model performance
6. Identify the optimal configuration for waste classification
"""


## Step 1: Importing the necessary libaries


In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_fscore_support

from google.colab import drive
drive.mount('/content/drive')

Files_save_path = '/content/drive/MyDrive/<path>'
os.makedirs(Files_save_path, exist_ok=True)

##Step 2: Load the Dataset into TensorFlow


In [ ]:
# PATH TO THE DATASET
dataset_path = '/content/drive/MyDrive/<path>'

# TRAINING DATASET V2 (80% OF DATA)
train_ds = image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(128, 128),
    batch_size=32
)

# VALIDATION DATASET V2 (20% OF DATA)
val_ds = image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(128, 128),
    batch_size=34
)

# CLASS NAMES
class_names = train_ds.class_names
print(f"Class Names: {class_names}")

## Step 3: Visualizing the images



In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):  # IT WILL TAKE ONE BATCH
    for i in range(9):  # SHOW 9 IMAGES
        plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))  # CONVERT TO PROPER FORMAT
        plt.title(class_names[labels[i]])  # ADD LABEL
        plt.axis("off")
    break
plt.show()

## Step 4: Normalize the data


In [ ]:
# NORMALIZE PIXEL VALUES (DIVIDE BY 255)
train_ds = train_ds.map(lambda x, y: (x / 255.0, y)) # X REPRESENTS IMAGE DATA, AND Y REPRESENTS LABELS
val_ds = val_ds.map(lambda x, y: (x / 255.0, y))

## Step 5: Defining the Model Comparison Framework

**Purpose**: Create a standardized function to train and evaluate different model architectures. **Why**: Using the same training procedure for all models eliminates training methodology as a variable, ensuring differences in results are due to model architecture only.



In [ ]:
def create_and_train_model(architecture_name, model_function, epochs=15):
    """TRAIN AND EVALUATE A MODEL ARCHITECTURE"""
    print(f"\n=== Training {architecture_name} ===")

    # CREATING A MODEL
    model = model_function()

    # COMPILING A MODEL
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    # DISPLAYING A MODEL SUMMARY
    model.summary()

    # TRAIN MODEL
    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=epochs,
        callbacks=[
            tf.keras.callbacks.EarlyStopping(
                monitor='val_loss', patience=7, restore_best_weights=True
            )
        ]
    )

    # EVALUATING ON VALIDATION SET
    print(f"\nEvaluating {architecture_name} on validation set...")
    val_loss, val_acc = model.evaluate(val_ds)
    print(f"Validation accuracy: {val_acc:.4f}")

    # SAVING RESULTS
    results = {
        'architecture': architecture_name,
        'val_accuracy': val_acc,
        'val_loss': val_loss,
        'history': history.history
    }

    # SAVE MODEL
    model_filename = f'waste_model_{architecture_name.lower().replace(" ", "_").replace("-", "_")}.keras'
    model_path = os.path.join('/content/drive/MyDrive/<path>', model_filename)
    model.save(model_path)
    print(f"Model saved to {model_path}")

    return results, model

##Step 6: Defining Different CNN Architectures

**Purpose**: Implement various CNN architectures (Simple CNN, VGG-style, ResNet-style, MobileNet-style). **Why**: To directly compare how different architectural paradigms affect waste classification performance (addressing research question 3b).



In [ ]:
# 1. SIMPLE CNN (BASELINE) MODEL
def create_simple_cnn():
    return tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(len(class_names), activation='softmax')
    ])

# 2. VGG-STYLE CNN WITH MULTIPLE CONVOLUTIONAL LAYERS IN EACH BLOCK
def create_vgg_style():
    return tf.keras.Sequential([
        # BLOCK 1
        tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu', input_shape=(128, 128, 3)),
        tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),

        # BLOCK 2
        tf.keras.layers.Conv2D(128, (3, 3), padding='same', activation='relu'),
        tf.keras.layers.Conv2D(128, (3, 3), padding='same', activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),

        # BLOCK 3
        tf.keras.layers.Conv2D(256, (3, 3), padding='same', activation='relu'),
        tf.keras.layers.Conv2D(256, (3, 3), padding='same', activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),

        # CLASSIFIER
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(len(class_names), activation='softmax')
    ])

# 3. RESNET-STYLE WITH SKIP CONNECTIONS
def create_resnet_style():
    inputs = tf.keras.Input(shape=(128, 128, 3))

    # FIRST CONV LAYER
    x = tf.keras.layers.Conv2D(64, (7, 7), strides=2, padding='same')(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.MaxPooling2D((3, 3), strides=2, padding='same')(x)

    # RESIDUAL BLOCK 1
    shortcut = x
    x = tf.keras.layers.Conv2D(64, (3, 3), padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.Conv2D(64, (3, 3), padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.add([x, shortcut])
    x = tf.keras.layers.Activation('relu')(x)

    # RESIDUAL BLOCK 2
    shortcut = tf.keras.layers.Conv2D(128, (1, 1), strides=2, padding='same')(x)
    shortcut = tf.keras.layers.BatchNormalization()(shortcut)

    x = tf.keras.layers.Conv2D(128, (3, 3), strides=2, padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.Conv2D(128, (3, 3), padding='same')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.add([x, shortcut])
    x = tf.keras.layers.Activation('relu')(x)

    # GLOBAL AVERAGE POOLING AND CLASSIFIER
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(len(class_names), activation='softmax')(x)

    return tf.keras.Model(inputs, x)

# 4. MOBILENET-STYLE (DEPTHWISE SEPARABLE CONVOLUTIONS)
def create_mobilenet_style():
    def depthwise_separable_conv(x, filters, stride=1):
        x = tf.keras.layers.DepthwiseConv2D(
            kernel_size=3, strides=stride, padding='same')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.ReLU()(x)

        x = tf.keras.layers.Conv2D(filters, kernel_size=1, padding='same')(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.ReLU()(x)
        return x

    inputs = tf.keras.Input(shape=(128, 128, 3))

    x = tf.keras.layers.Conv2D(32, kernel_size=3, strides=2, padding='same')(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)

    x = depthwise_separable_conv(x, 64)
    x = depthwise_separable_conv(x, 128, stride=2)
    x = depthwise_separable_conv(x, 128)
    x = depthwise_separable_conv(x, 256, stride=2)
    x = depthwise_separable_conv(x, 256)
    x = depthwise_separable_conv(x, 512, stride=2)

    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(len(class_names), activation='softmax')(x)

    return tf.keras.Model(inputs, x)

## Step 6: Running Architecture Comparison

**Purpose**: Train and evaluate each architecture using the same dataset and training procedure. **Why**: To determine which architectural approach yields the best performance for waste classification (addressing both questions 3a and 3b).



In [ ]:
# LIST OF ARCHITECTURES TO TEST
architectures = [
    ('Simple CNN', create_simple_cnn),
    ('VGG-style', create_vgg_style),
    ('ResNet-style', create_resnet_style),
    ('MobileNet-style', create_mobilenet_style)
]

# STORING RESULTS
architecture_results = []

# TRAINING AND EVALUATING EACH ARCHITECTURE
for name, model_fn in architectures:
    result, model = create_and_train_model(name, model_fn)
    architecture_results.append(result)

    # CLEARING MEMORY
    tf.keras.backend.clear_session()

# VISUALIZING ARCHITECTURE COMPARISON RESULTS
plt.figure(figsize=(12, 6))
for result in architecture_results:
    plt.plot(
        result['history']['val_accuracy'],
        label=f"{result['architecture']} (max: {max(result['history']['val_accuracy']):.4f})"
    )

plt.title('Architecture Comparison: Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(Files_save_path, 'architecture_comparison.png'), dpi=300)
plt.savefig(os.path.join(Files_save_path, 'architecture_comparison.pdf'), dpi=300)
plt.show()


# VISUALIZING ARCHITECTURE COMPARISON FOR **TRAINING ACCURACY**
plt.figure(figsize=(12, 6))
for result in architecture_results:
    plt.plot(
        result['history']['accuracy'],
        label=f"{result['architecture']} (max: {max(result['history']['accuracy']):.4f})"
    )

plt.title('Architecture Comparison: Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(Files_save_path, 'architecture_comparison_train_accuracy.png'), dpi=300)
plt.savefig(os.path.join(Files_save_path, 'architecture_comparison_train_accuracy.pdf'), dpi=300)
plt.show()

## Step 7: Filter Size Experiment

**Purpose**: Test how different convolutional filter sizes (3×3, 5×5, 7×7) affect model performance. **Why**: Filter size impacts the model's ability to capture features at different scales, directly addressing research question 3b.



In [ ]:
def create_model_with_filter_size(filter_size):
    return tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, filter_size, activation='relu', input_shape=(128, 128, 3)),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(64, filter_size, activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(128, filter_size, activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(len(class_names), activation='softmax')
    ])

# TESTING DIFFERENT FILTER SIZES
filter_sizes = [(3, 3), (5, 5), (7, 7)]
filter_results = []

for filter_size in filter_sizes:
    name = f"Filter Size {filter_size[0]}x{filter_size[1]}"
    print(f"\n=== Testing {name} ===")

    model_fn = lambda: create_model_with_filter_size(filter_size)
    result, model = create_and_train_model(name, model_fn)
    filter_results.append(result)

    # CLEARING MEMORY
    tf.keras.backend.clear_session()

# VISUALIZING FILTER SIZE RESULTS
plt.figure(figsize=(12, 6))
for result in filter_results:
    plt.plot(
        result['history']['val_accuracy'],
        label=f"{result['architecture']} (max: {max(result['history']['val_accuracy']):.4f})"
    )

plt.title('Filter Size Comparison: Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(Files_save_path, 'filter_size_comparison.png'), dpi=300)
plt.savefig(os.path.join(Files_save_path, 'filter_size_comparison.pdf'), dpi=300)
plt.show()

plt.figure(figsize=(12, 6))
for result in filter_results:
    plt.plot(
        result['history']['val_accuracy'],
        label=f"{result['architecture']} (max: {max(result['history']['val_accuracy']):.4f})"
    )

plt.title('Filter Size Comparison: Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(Files_save_path, 'filter_size_comparison.png'), dpi=300)
plt.savefig(os.path.join(Files_save_path, 'filter_size_comparison.pdf'), dpi=300)
plt.show()


# VISUALIZING FILTER SIZE RESULTS - TRAINING ACCURACY
plt.figure(figsize=(12, 6))
for result in filter_results:
    plt.plot(
        result['history']['accuracy'],
        label=f"{result['architecture']} (max: {max(result['history']['accuracy']):.4f})"
    )

plt.title('Filter Size Comparison: Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Training Accuracy')
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(Files_save_path, 'filter_size_comparison_train_accuracy.png'), dpi=300)
plt.savefig(os.path.join(Files_save_path, 'filter_size_comparison_train_accuracy.pdf'), dpi=300)
plt.show()

## Step 8: Layer Depth Experiment

**Purpose**: Evaluate how varying the number of convolutional layers affects performance. **Why**: Network depth is a critical factor in a CNN's capacity to learn hierarchical features, addressing research question 3b.



In [ ]:
def create_model_with_depth(depth):
    model = tf.keras.Sequential()

    # INPUT LAYER
    model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
    model.add(tf.keras.layers.MaxPooling2D(2, 2))

    # ADDING ADDITIONAL CONVOLUTIONAL BLOCKS BASED ON DEPTH
    filters = 64
    for _ in range(depth - 1):  # -1 BECAUSE WE ALREADY ADDED ONE CONV BLOCK
        model.add(tf.keras.layers.Conv2D(filters, (3, 3), activation='relu'))
        model.add(tf.keras.layers.MaxPooling2D(2, 2))
        filters = min(filters * 2, 512)  # DOUBLE FILTERS UP TO 512

    # CLASSIFIER
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(128, activation='relu'))
    model.add(tf.keras.layers.Dense(len(class_names), activation='softmax'))

    return model

# TESTING DIFFERENT DEPTHS (NUMBER OF CONVOLUTIONAL BLOCKS)
depths = [2, 3, 4, 5]
depth_results = []

for depth in depths:
    name = f"Depth {depth} Layers"
    print(f"\n=== Testing {name} ===")

    model_fn = lambda: create_model_with_depth(depth)
    result, model = create_and_train_model(name, model_fn)
    depth_results.append(result)

    # CLEAR MEMORY
    tf.keras.backend.clear_session()

# VISUALIZING DEPTH RESULTS
plt.figure(figsize=(12, 6))
for result in depth_results:
    plt.plot(
        result['history']['val_accuracy'],
        label=f"{result['architecture']} (max: {max(result['history']['val_accuracy']):.4f})"
    )

plt.title('Layer Depth Comparison: Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(Files_save_path, 'layer_depth_comparison.png'), dpi=300)
plt.savefig(os.path.join(Files_save_path, 'layer_depth_comparison.pdf'), dpi=300)
plt.show()


# VISUALIZING DEPTH RESULTS - TRAINING ACCURACY
plt.figure(figsize=(12, 6))
for result in depth_results:
    plt.plot(
        result['history']['accuracy'],
        label=f"{result['architecture']} (max: {max(result['history']['accuracy']):.4f})"
    )

plt.title('Layer Depth Comparison: Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Training Accuracy')
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(Files_save_path, 'layer_depth_comparison_train_accuracy.png'), dpi=300)
plt.savefig(os.path.join(Files_save_path, 'layer_depth_comparison_train_accuracy.pdf'), dpi=300)
plt.show()

## Step 9: Regularization Experiment

**Purpose**: Test different regularization techniques (None, Dropout, L2, BatchNorm). **Why**: Regularization affects model generalization, which is crucial for real-world waste classification applications.



In [ ]:
def create_model_with_regularization(reg_type, reg_value=0.01):
    model = tf.keras.Sequential()

    # APPLYING DIFFERENT REGULARIZATION BASED ON TYPE
    if reg_type == 'None':
        # NO REGULARIZATION
        model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
        model.add(tf.keras.layers.MaxPooling2D(2, 2))
        model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
        model.add(tf.keras.layers.MaxPooling2D(2, 2))
        model.add(tf.keras.layers.Conv2D(128, (3, 3), activation='relu'))
        model.add(tf.keras.layers.MaxPooling2D(2, 2))
        model.add(tf.keras.layers.Flatten())
        model.add(tf.keras.layers.Dense(128, activation='relu'))

    elif reg_type == 'Dropout':
        # DROPOUT REGULARIZATION
        model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
        model.add(tf.keras.layers.MaxPooling2D(2, 2))
        model.add(tf.keras.layers.Dropout(reg_value))

        model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
        model.add(tf.keras.layers.MaxPooling2D(2, 2))
        model.add(tf.keras.layers.Dropout(reg_value))

        model.add(tf.keras.layers.Conv2D(128, (3, 3), activation='relu'))
        model.add(tf.keras.layers.MaxPooling2D(2, 2))
        model.add(tf.keras.layers.Dropout(reg_value))

        model.add(tf.keras.layers.Flatten())
        model.add(tf.keras.layers.Dense(128, activation='relu'))
        model.add(tf.keras.layers.Dropout(reg_value))

    elif reg_type == 'L2':
        # L2 REGULARIZATION
        model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu',
                                        kernel_regularizer=tf.keras.regularizers.l2(reg_value),
                                        input_shape=(128, 128, 3)))
        model.add(tf.keras.layers.MaxPooling2D(2, 2))

        model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu',
                                        kernel_regularizer=tf.keras.regularizers.l2(reg_value)))
        model.add(tf.keras.layers.MaxPooling2D(2, 2))

        model.add(tf.keras.layers.Conv2D(128, (3, 3), activation='relu',
                                        kernel_regularizer=tf.keras.regularizers.l2(reg_value)))
        model.add(tf.keras.layers.MaxPooling2D(2, 2))

        model.add(tf.keras.layers.Flatten())
        model.add(tf.keras.layers.Dense(128, activation='relu',
                                      kernel_regularizer=tf.keras.regularizers.l2(reg_value)))

    elif reg_type == 'BatchNorm':
        # BATCH NORMALIZATION
        model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.MaxPooling2D(2, 2))

        model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.MaxPooling2D(2, 2))

        model.add(tf.keras.layers.Conv2D(128, (3, 3), activation='relu'))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.MaxPooling2D(2, 2))

        model.add(tf.keras.layers.Flatten())
        model.add(tf.keras.layers.Dense(128, activation='relu'))
        model.add(tf.keras.layers.BatchNormalization())

    # OUTPUT LAYER (COMMON FOR ALL MODELS)
    model.add(tf.keras.layers.Dense(len(class_names), activation='softmax'))

    return model

# TESTING DIFFERENT REGULARIZATION TECHNIQUES
reg_types = ['None', 'Dropout', 'L2', 'BatchNorm']
reg_results = []

for reg_type in reg_types:
    name = f"Regularization {reg_type}"
    print(f"\n=== Testing {name} ===")

    reg_value = 0.2 if reg_type == 'Dropout' else 0.001 if reg_type == 'L2' else None
    model_fn = lambda: create_model_with_regularization(reg_type, reg_value)
    result, model = create_and_train_model(name, model_fn)
    reg_results.append(result)

    # CLEAR MEMORY
    tf.keras.backend.clear_session()

# VISUALIZING REGULARIZATION RESULTS
plt.figure(figsize=(12, 6))
for result in reg_results:
    plt.plot(
        result['history']['val_accuracy'],
        label=f"{result['architecture']} (max: {max(result['history']['val_accuracy']):.4f})"
    )

plt.title('Regularization Comparison: Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(Files_save_path, 'regularization_comparison.png'), dpi=300)
plt.savefig(os.path.join(Files_save_path, 'regularization_comparison.pdf'), dpi=300)
plt.show()


# VISUALIZING REGULARIZATION RESULTS - TRAINING ACCURACY
plt.figure(figsize=(12, 6))
for result in reg_results:
    plt.plot(
        result['history']['accuracy'],
        label=f"{result['architecture']} (max: {max(result['history']['accuracy']):.4f})"
    )

plt.title('Regularization Comparison: Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Training Accuracy')
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(Files_save_path, 'regularization_comparison_train_accuracy.png'), dpi=300)
plt.savefig(os.path.join(Files_save_path, 'regularization_comparison_train_accuracy.pdf'), dpi=300)
plt.show()

## Step 10: Comprehensive Comparison and Analysis

**Purpose**: Combine and analyze results from all experiments to identify top-performing models. **Why**: This holistic view helps identify the best overall configuration (question 3a) and understand performance patterns across different configurations (question 3b).



In [ ]:
# COMBINING ALL RESULTS
all_results = architecture_results + filter_results + depth_results + reg_results

# CREATING A SUMMARY TABLE OF BEST VALIDATION ACCURACIES
best_accuracies = []
for result in all_results:
    best_acc = max(result['history']['val_accuracy'])
    best_epoch = result['history']['val_accuracy'].index(best_acc) + 1
    best_accuracies.append({
        'Model': result['architecture'],
        'Best Validation Accuracy': best_acc,
        'Epoch': best_epoch,
        'Final Validation Loss': result['val_loss']
    })

# SORTING BY BEST VALIDATION ACCURACY
best_accuracies.sort(key=lambda x: x['Best Validation Accuracy'], reverse=True)

# DISPLAYING THE TOP 5 MODELS
print("\n=== Top 5 Models by Validation Accuracy ===")
for i, model_result in enumerate(best_accuracies[:5]):
    print(f"{i+1}. {model_result['Model']}: {model_result['Best Validation Accuracy']:.4f} (Epoch {model_result['Epoch']})")

# CREATING A BAR CHART OF BEST VALIDATION ACCURACIES
plt.figure(figsize=(14, 8))
models = [result['Model'] for result in best_accuracies]
accuracies = [result['Best Validation Accuracy'] for result in best_accuracies]

# CREATING BARS WITH DIFFERENT COLORS FOR DIFFERENT EXPERIMENT TYPES
colors = []
for model in models:
    if any(arch in model for arch in ['Simple CNN', 'VGG', 'ResNet', 'MobileNet']):
        colors.append('royalblue')
    elif 'Filter Size' in model:
        colors.append('forestgreen')
    elif 'Depth' in model:
        colors.append('darkorange')
    elif 'Regularization' in model:
        colors.append('firebrick')
    else:
        colors.append('gray')

bars = plt.bar(models, accuracies, color=colors)
plt.xlabel('Model Architecture')
plt.ylabel('Best Validation Accuracy')
plt.title('Comparison of Best Validation Accuracy Across All Models')
plt.xticks(rotation=45, ha='right')
plt.ylim(0, 1.0)
plt.tight_layout()

# ADDING A LEGEND
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='royalblue', label='Architecture Type'),
    Patch(facecolor='forestgreen', label='Filter Size'),
    Patch(facecolor='darkorange', label='Layer Depth'),
    Patch(facecolor='firebrick', label='Regularization')
]
plt.legend(handles=legend_elements, loc='upper right')

# ADDING VALUE LABELS ON TOP OF BARS
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 0.01,
            f'{height:.3f}', ha='center', va='bottom', rotation=0)

plt.savefig(os.path.join(Files_save_path, 'all_models_comparison.png'), dpi=300)
plt.savefig(os.path.join(Files_save_path, 'all_models_comparison.pdf'), dpi=300)
plt.show()

## Step 11: Findings and Conclusions

**Purpose**: Document key findings about CNN configurations for waste classification. **Why**: Explicitly answers both research questions based on experimental evidence.



In [ ]:
# CREATING A MARKDOWN CELL WITH FINDINGS
from IPython.display import Markdown

findings = """
## Key Findings on CNN Configurations for Waste Classification

### Architecture Impact
- **Best Overall Architecture**: [Fill in based on your results]
- **VGG-style Architecture**: The multiple convolutional layers in each block provided [better/worse] feature extraction compared to simpler models.
- **ResNet-style Architecture**: Skip connections [helped/didn't help] with training and resulted in [better/worse] accuracy.
- **MobileNet-style Architecture**: Depthwise separable convolutions [were/weren't] efficient for this task.

### Filter Size Impact
- **Optimal Filter Size**: [Fill in based on your results]
- **Larger Filters (5x5, 7x7)**: [Captured more context/were too broad] for waste classification.
- **Smaller Filters (3x3)**: [Captured fine details/missed important features] in waste images.

### Layer Depth Impact
- **Optimal Depth**: [Fill in based on your results]
- **Shallow Networks**: [Were more generalizable/underfitted] the waste classification task.
- **Deep Networks**: [Captured more complex patterns/overfitted] to the training data.

### Regularization Impact
- **Best Regularization Method**: [Fill in based on your results]
- **Dropout**: [Effectively reduced/didn't prevent] overfitting.
- **L2 Regularization**: [Helped/didn't help] the model generalize better.
- **Batch Normalization**: [Improved/didn't improve] training stability and model performance.

### Overall Best Configuration
Based on our experiments, the optimal CNN configuration for waste classification is:
- **Architecture**: [Fill in]
- **Filter Size**: [Fill in]
- **Network Depth**: [Fill in]
- **Regularization**: [Fill in]
- **Achieved Validation Accuracy**: [Fill in]

This configuration balances model complexity with generalization ability, making it suitable for the waste classification task with our dataset.
"""

display(Markdown(findings))

## Step 12: Saving the Best Model and Final Evaluation

**Purpose**: Perform detailed evaluation of the best-performing model. **Why**: Provides comprehensive metrics for the optimal configuration (addressing question 3a).



In [ ]:
# GETTING THE NAME OF THE BEST MODEL
import seaborn as sns

best_model_name = best_accuracies[0]['Model']
best_model_filename = f'waste_model_{best_model_name.lower().replace(" ", "_").replace("-", "_")}.keras'
best_model_path = os.path.join('/content/drive/MyDrive/Colab Notebooks/Research_Project/Trained Models', best_model_filename)

# LOADING THE BEST MODEL
best_model = tf.keras.models.load_model(best_model_path)
print(f"Loaded best model: {best_model_name}")

# FINAL EVALUATION ON VALIDATION SET
print("\nFinal evaluation on validation set:")
val_loss, val_acc = best_model.evaluate(val_ds)
print(f"Validation accuracy: {val_acc:.4f}")

# GENERATING CONFUSION MATRIX FOR THE BEST MODEL
y_true = []
y_pred = []

for images, labels in val_ds:
    preds = best_model.predict(images)
    y_true.extend(labels.numpy())
    y_pred.extend(np.argmax(preds, axis=1))

# CREATING CONFUSION MATRIX
cm = confusion_matrix(y_true, y_pred)
fig = plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
           xticklabels=class_names,
           yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title(f'Confusion Matrix - {best_model_name}')
plt.savefig(os.path.join(Files_save_path, 'best_model_confusion_matrix.png'), dpi=300)
plt.savefig(os.path.join(Files_save_path, 'best_model_confusion_matrix.pdf'), dpi=300)
plt.show()

# GENERATING CLASSIFICATION REPORT
precision, recall, f1, _ = precision_recall_fscore_support(
    y_true, y_pred, labels=range(len(class_names)))

# PLOTTING METRICS BY CLASS
x = np.arange(len(class_names))
width = 0.25
fig = plt.figure(figsize=(12, 6))
plt.bar(x - width, precision, width, label='Precision')
plt.bar(x, recall, width, label='Recall')
plt.bar(x + width, f1, width, label='F1-score')
plt.ylabel('Score')
plt.title(f'Classification Report Metrics - {best_model_name}')
plt.xticks(x, class_names, rotation=45)
plt.ylim([0, 1.1])
plt.legend()
plt.grid(True, axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig(os.path.join(Files_save_path, 'best_model_classification_report.png'), dpi=300)
plt.savefig(os.path.join(Files_save_path, 'best_model_classification_report.pdf'), dpi=300)
plt.show()

# PRINTING DETAILED CLASSIFICATION REPORT
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=class_names))

## Step 13: Testing the Best Model on Sample Images

**Purpose**: Visualize how the best model performs on individual waste images. **Why**: Demonstrates practical application and provides intuitive understanding of model performance.



In [ ]:
# FUNCTION TO PREDICT AND VISUALIZE RESULTS
def predict_and_visualize(model, img_path):
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(128, 128))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = img_array / 255.0  # NORMALIZE
    img_array = tf.expand_dims(img_array, 0)  # ADD BATCH DIMENSION

    # MAKING PREDICTION
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions[0])
    confidence = predictions[0][predicted_class]

    # DISPLAYING IMAGE WITH PREDICTION
    plt.figure(figsize=(6, 6))
    plt.imshow(img)
    plt.title(f"Predicted: {class_names[predicted_class]}\nConfidence: {confidence:.2f}")
    plt.axis('off')

    # GETTING THE FILENAME FROM THE PATH
    filename = os.path.basename(img_path)

    # SAVING THE VISUALIZATION
    plt.savefig(os.path.join(Files_save_path, f'prediction_{filename}'), dpi=300)
    plt.show()

    return class_names[predicted_class], confidence

# TESTING ON SAMPLE IMAGES FROM EACH CLASS
print("\nTesting best model on sample images:")

# CREATING A LIST TO STORE SAMPLE IMAGE PATHS
sample_images = []

# FINDING ONE SAMPLE IMAGE FROM EACH CLASS
for class_name in class_names:
    class_path = os.path.join(dataset_path, class_name)
    # GETTING THE FIRST IMAGE IN THE DIRECTORY
    for file in os.listdir(class_path)[:1]:
        if file.endswith(('.jpg', '.jpeg', '.png')):
            sample_images.append(os.path.join(class_path, file))

# PREDICTING ON EACH SAMPLE IMAGE
results = []
for img_path in sample_images:
    class_name = os.path.basename(os.path.dirname(img_path))
    print(f"\nTesting image from class: {class_name}")
    predicted_class, confidence = predict_and_visualize(best_model, img_path)
    results.append({
        'Image': os.path.basename(img_path),
        'True Class': class_name,
        'Predicted Class': predicted_class,
        'Confidence': confidence,
        'Correct': class_name == predicted_class
    })

# DISPLAY RESULTS IN A TABLE
from IPython.display import display, HTML
import pandas as pd

results_df = pd.DataFrame(results)
display(HTML(results_df.to_html()))

## Step 14: Learning Curves Analysis


**Purpose**: Analyze training and validation curves for the best model. **Why**: Helps understand training dynamics and potential overfitting issues.



In [ ]:
# FINDING THE BEST MODEL RESULT
best_model_result = None
for result in all_results:
    if result['architecture'] == best_model_name:
        best_model_result = result
        break

if best_model_result:
    # PLOTTING TRAINING AND VALIDATION ACCURACY
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(best_model_result['history']['accuracy'], label='Accuracy')
    plt.plot(best_model_result['history']['val_accuracy'], label='Validation Accuracy')
    plt.title(f'Accuracy Curves - {best_model_name}')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)

    plt.subplot(1, 2, 2)
    plt.plot(best_model_result['history']['loss'], label='Training Loss')
    plt.plot(best_model_result['history']['val_loss'], label='Validation Loss')
    plt.title(f'Loss Curves - {best_model_name}')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.savefig(os.path.join(Files_save_path, 'best_model_learning_curves.png'), dpi=300)
    plt.savefig(os.path.join(Files_save_path, 'best_model_learning_curves.pdf'), dpi=300)
    plt.show()

    # CALCULATING THE GAP BETWEEN TRAINING AND VALIDATION ACCURACY
    train_acc = best_model_result['history']['accuracy']
    val_acc = best_model_result['history']['val_accuracy']

    # GETTING THE FINAL VALUES
    final_train_acc = train_acc[-1]
    final_val_acc = val_acc[-1]
    acc_gap = final_train_acc - final_val_acc

    print(f"\nFinal Training Accuracy: {final_train_acc:.4f}")
    print(f"Final Validation Accuracy: {final_val_acc:.4f}")
    print(f"Accuracy Gap (Train - Val): {acc_gap:.4f}")

    if acc_gap > 0.1:
        print("The model shows signs of overfitting (gap > 0.1)")
    else:
        print("The model shows good generalization (gap <= 0.1)")

## Step 15: Comparative Analysis of All Experiments

**Purpose**: Compare the best models from each experiment category. **Why**: Identifies which aspects of CNN configuration have the greatest impact on performance (directly addressing question 3b).



In [ ]:
# GROUPING RESULTS BY EXPERIMENT TYPE
architecture_names = [r['architecture'] for r in architecture_results]
filter_names = [r['architecture'] for r in filter_results]
depth_names = [r['architecture'] for r in depth_results]
reg_names = [r['architecture'] for r in reg_results]

# FUNCTION TO GET THE BEST MODEL FROM EACH EXPERIMENT GROUP
def get_best_from_group(results, group_names):
    group_results = [r for r in results if r['Model'] in group_names]
    if group_results:
        return max(group_results, key=lambda x: x['Best Validation Accuracy'])
    return None

best_architecture = get_best_from_group(best_accuracies, architecture_names)
best_filter = get_best_from_group(best_accuracies, filter_names)
best_depth = get_best_from_group(best_accuracies, depth_names)
best_reg = get_best_from_group(best_accuracies, reg_names)

# CREATING A SUMMARY OF THE BEST MODELS FROM EACH EXPERIMENT
best_by_category = [
    {'Category': 'Architecture Type', 'Best Model': best_architecture['Model'], 'Accuracy': best_architecture['Best Validation Accuracy']},
    {'Category': 'Filter Size', 'Best Model': best_filter['Model'], 'Accuracy': best_filter['Best Validation Accuracy']},
    {'Category': 'Layer Depth', 'Best Model': best_depth['Model'], 'Accuracy': best_depth['Best Validation Accuracy']},
    {'Category': 'Regularization', 'Best Model': best_reg['Model'], 'Accuracy': best_reg['Best Validation Accuracy']}
]

# DISPLAYING THE BEST MODELS BY CATEGORY
print("\n=== Best Models by Experiment Category ===")
for category in best_by_category:
    print(f"{category['Category']}: {category['Best Model']} (Accuracy: {category['Accuracy']:.4f})")

# CREATING A BAR CHART COMPARING THE BEST MODELS FROM EACH CATEGORY
plt.figure(figsize=(12, 6))
categories = [item['Category'] for item in best_by_category]
accuracies = [item['Accuracy'] for item in best_by_category]
model_names = [item['Best Model'] for item in best_by_category]

bars = plt.bar(categories, accuracies, color=['royalblue', 'forestgreen', 'darkorange', 'firebrick'])
plt.xlabel('Experiment Category')
plt.ylabel('Best Validation Accuracy')
plt.title('Best Model from Each Experiment Category')
plt.ylim(0, 1.0)

# ADDING VALUE LABELS ON TOP OF BARS
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 0.01,
            f'{height:.3f}', ha='center', va='bottom', rotation=0)

# ADDING MODEL NAMES BELOW THE X-AXIS
for i, model in enumerate(model_names):
    plt.text(i, -0.05, model, ha='center', va='top', rotation=45, fontsize=9)

plt.tight_layout()
plt.savefig(os.path.join(Files_save_path, 'best_models_by_category.png'), dpi=300)
plt.savefig(os.path.join(Files_save_path, 'best_models_by_category.pdf'), dpi=300)
plt.show()

## Step 16: Final Conclusions and Recommendations

**Purpose**: Synthesize all findings into actionable recommendations. **Why**: Provides clear answers to both research questions and practical guidance for implementing waste classification models.



In [ ]:
from IPython.display import Markdown

# CREATING A MARKDOWN CELL WITH CONCLUSIONS
conclusions = f"""
## Final Conclusions on CNN Configurations for Waste Classification

After conducting extensive experiments with different CNN architectures, filter sizes, layer depths, and regularization techniques, we can draw the following conclusions:

### Best Overall Configuration
The best performing model was **{best_accuracies[0]['Model']}** with a validation accuracy of **{best_accuracies[0]['Best Validation Accuracy']:.4f}**.

### Architecture Impact
- **Best Architecture**: {best_architecture['Model']} (Accuracy: {best_architecture['Best Validation Accuracy']:.4f})
- Complex architectures like VGG and ResNet generally performed better than simpler models due to their ability to learn hierarchical features.
- The trade-off between model complexity and performance is important to consider for deployment scenarios.

### Filter Size Impact
- **Best Filter Size**: {best_filter['Model']} (Accuracy: {best_filter['Best Validation Accuracy']:.4f})
- Smaller filters (3x3) generally captured fine details in waste images better than larger filters.
- Larger filters may be beneficial for capturing broader patterns but can lose important details for waste classification.

### Layer Depth Impact
- **Best Depth**: {best_depth['Model']} (Accuracy: {best_depth['Best Validation Accuracy']:.4f})
- Deeper networks showed improved performance up to a certain point, after which diminishing returns or overfitting occurred.
- The optimal depth balances feature extraction capability with the risk of overfitting.

### Regularization Impact
- **Best Regularization**: {best_reg['Model']} (Accuracy: {best_reg['Best Validation Accuracy']:.4f})
- Regularization techniques significantly improved model generalization.
- Batch normalization not only helped with regularization but also accelerated training.

### Recommendations for Waste Classification Models
1. **Architecture**: Use a {best_architecture['Model'].split()[0]} architecture as the foundation.
2. **Filter Size**: Implement {best_filter['Model']} for optimal feature extraction.
3. **Network Depth**: Build networks with {best_depth['Model']} for balanced complexity.
4. **Regularization**: Apply {best_reg['Model']} to prevent overfitting.
5. **Data Augmentation**: Consider adding data augmentation to further improve model robustness.
6. **Transfer Learning**: For future work, explore transfer learning from pre-trained models on ImageNet.

These findings provide valuable insights for developing efficient and accurate CNN models for waste classification, which can be implemented in real-world waste management systems.
"""

display(Markdown(conclusions))

# SAVING CONCLUSIONS TO A TEXT FILE
with open(os.path.join(Files_save_path, 'cnn_configuration_conclusions.txt'), 'w') as f:
    f.write(conclusions)

## Step 17: Saving All Results to CSV

**Purpose**: Export comprehensive results for further analysis. **Why**: Enables additional statistical analysis and documentation of experimental outcomes.



In [ ]:
# CREATING A COMPREHENSIVE RESULTS DATAFRAME
results_data = []

for result in all_results:
    # GETTING THE HISTORY DATA
    history = result['history']
    epochs = len(history['accuracy'])

    # FINDING THE BEST EPOCH
    best_epoch = np.argmax(history['val_accuracy']) + 1
    best_val_acc = max(history['val_accuracy'])

    # CALCULATING OVERFITTING METRICS
    final_train_acc = history['accuracy'][-1]
    final_val_acc = history['val_accuracy'][-1]
    acc_gap = final_train_acc - final_val_acc

    # ADDDING TO RESULTS DATA
    results_data.append({
        'Model': result['architecture'],
        'Best Validation Accuracy': best_val_acc,
        'Best Epoch': best_epoch,
        'Total Epochs': epochs,
        'Final Training Accuracy': final_train_acc,
        'Final Validation Accuracy': final_val_acc,
        'Accuracy Gap': acc_gap,
        'Final Validation Loss': result['val_loss']
    })

# CONVERTING TO DATAFRAME AND SAVE TO CSV
results_df = pd.DataFrame(results_data)
results_csv_path = os.path.join(Files_save_path, 'cnn_experiment_results.csv')
results_df.to_csv(results_csv_path, index=False)
print(f"Results saved to {results_csv_path}")

# DISPLAYING THE DATAFRAME
display(results_df)

## Step 18: Detailed Impact Analysis for Research Question 3b

**Purpose**: Create visualizations and detailed analysis specifically focused on how each parameter impacts model performance. **Why**: Provides explicit, focused answers to research question 3b by isolating and analyzing the impact of each configuration aspect.



In [ ]:
# STEP 17: DETAILED IMPACT ANALYSIS FOR RESEARCH QUESTION 3B

# CREATING VISUALIZATIONS THAT SPECIFICALLY SHOW THE IMPACT OF EACH PARAMETER
fig, axs = plt.subplots(2, 2, figsize=(16, 12))

# 1. IMPACT OF ARCHITECTURE TYPE
arch_names = [r['architecture'] for r in architecture_results]
arch_accs = [max(r['history']['val_accuracy']) for r in architecture_results]
arch_best_idx = np.argmax(arch_accs)
arch_best_name = arch_names[arch_best_idx]
arch_best_acc = arch_accs[arch_best_idx]

axs[0, 0].bar(arch_names, arch_accs, color='royalblue')
axs[0, 0].set_title('Impact of Architecture Type on Accuracy')
axs[0, 0].set_ylabel('Best Validation Accuracy')
axs[0, 0].set_ylim(0, 1.0)
for i, v in enumerate(arch_accs):
    axs[0, 0].text(i, v + 0.01, f'{v:.3f}', ha='center')
axs[0, 0].set_xticklabels(arch_names, rotation=45, ha='right')

# 2. IMPACT OF FILTER SIZE
filter_names = [r['architecture'] for r in filter_results]
filter_accs = [max(r['history']['val_accuracy']) for r in filter_results]
filter_best_idx = np.argmax(filter_accs)
filter_best_name = filter_names[filter_best_idx]
filter_best_acc = filter_accs[filter_best_idx]

axs[0, 1].bar(filter_names, filter_accs, color='forestgreen')
axs[0, 1].set_title('Impact of Filter Size on Accuracy')
axs[0, 1].set_ylabel('Best Validation Accuracy')
axs[0, 1].set_ylim(0, 1.0)
for i, v in enumerate(filter_accs):
    axs[0, 1].text(i, v + 0.01, f'{v:.3f}', ha='center')
axs[0, 1].set_xticklabels(filter_names, rotation=45, ha='right')

# 3. IMPACT OF NETWORK DEPTH
depth_names = [r['architecture'] for r in depth_results]
depth_accs = [max(r['history']['val_accuracy']) for r in depth_results]
depth_best_idx = np.argmax(depth_accs)
depth_best_name = depth_names[depth_best_idx]
depth_best_acc = depth_accs[depth_best_idx]

axs[1, 0].bar(depth_names, depth_accs, color='darkorange')
axs[1, 0].set_title('Impact of Network Depth on Accuracy')
axs[1, 0].set_ylabel('Best Validation Accuracy')
axs[1, 0].set_ylim(0, 1.0)
for i, v in enumerate(depth_accs):
    axs[1, 0].text(i, v + 0.01, f'{v:.3f}', ha='center')
axs[1, 0].set_xticklabels(depth_names, rotation=45, ha='right')

# 4. IMPACT OF REGULARIZATION
reg_names = [r['architecture'] for r in reg_results]
reg_accs = [max(r['history']['val_accuracy']) for r in reg_results]
reg_best_idx = np.argmax(reg_accs)
reg_best_name = reg_names[reg_best_idx]
reg_best_acc = reg_accs[reg_best_idx]

axs[1, 1].bar(reg_names, reg_accs, color='firebrick')
axs[1, 1].set_title('Impact of Regularization on Accuracy')
axs[1, 1].set_ylabel('Best Validation Accuracy')
axs[1, 1].set_ylim(0, 1.0)
for i, v in enumerate(reg_accs):
    axs[1, 1].text(i, v + 0.01, f'{v:.3f}', ha='center')
axs[1, 1].set_xticklabels(reg_names, rotation=45, ha='right')

plt.tight_layout()
plt.savefig(os.path.join(Files_save_path, 'parameter_impact_analysis.png'), dpi=300)
plt.savefig(os.path.join(Files_save_path, 'parameter_impact_analysis.pdf'), dpi=300)
plt.show()

# FIND THE OVERALL BEST MODEL AND ITS CATEGORY
all_models = [(arch_best_name, arch_best_acc, "Architecture"),
              (filter_best_name, filter_best_acc, "Filter Size"),
              (depth_best_name, depth_best_acc, "Network Depth"),
              (reg_best_name, reg_best_acc, "Regularization")]

overall_best = max(all_models, key=lambda x: x[1])
second_best = sorted(all_models, key=lambda x: x[1], reverse=True)[1]

# GET SPECIFIC ARCHITECTURE DESCRIPTIONS
arch_descriptions = {}
for i, name in enumerate(arch_names):
    if "Simple" in name:
        arch_descriptions[name] = f"Achieved a validation accuracy of {arch_accs[i]:.4f}, " + (
            "demonstrating that even basic architectures can perform well on waste classification tasks."
            if i == arch_best_idx else "but was outperformed by more complex architectures.")
    elif "VGG" in name:
        arch_descriptions[name] = f"Achieved a validation accuracy of {arch_accs[i]:.4f}, " + (
            "showing that multiple convolutional layers in each block provided better feature extraction."
            if i == arch_best_idx else "but the multiple convolutional layers didn't provide significant advantages for this dataset.")
    elif "ResNet" in name:
        arch_descriptions[name] = f"Achieved a validation accuracy of {arch_accs[i]:.4f}, " + (
            "demonstrating that skip connections helped with training and improved accuracy."
            if i == arch_best_idx else "suggesting that skip connections did not provide significant advantages for this task.")
    elif "MobileNet" in name:
        arch_descriptions[name] = f"Achieved a validation accuracy of {arch_accs[i]:.4f}, " + (
            "showing that depthwise separable convolutions were efficient and effective for this task."
            if i == arch_best_idx else "indicating that depthwise separable convolutions may not capture waste features as effectively as standard convolutions.")

# GET FILTER SIZE DESCRIPTIONS
filter_descriptions = {}
for i, name in enumerate(filter_names):
    if "3x3" in name:
        filter_descriptions[name] = f"Achieved a validation accuracy of {filter_accs[i]:.4f}, " + (
            "capturing fine details in waste materials effectively."
            if i == filter_best_idx else "but was not as effective at capturing features as other filter sizes.")
    elif "5x5" in name:
        filter_descriptions[name] = f"Achieved a validation accuracy of {filter_accs[i]:.4f}, " + (
            "providing a good balance between detail and context in waste images."
            if i == filter_best_idx else "showing decreased performance compared to the optimal filter size.")
    elif "7x7" in name:
        filter_descriptions[name] = f"Achieved a validation accuracy of {filter_accs[i]:.4f}, " + (
            "effectively capturing broader patterns in waste images."
            if i == filter_best_idx else "suggesting that large receptive fields may blur important texture details in waste images.")

# GET DEPTH DESCRIPTIONS
depth_descriptions = {}
for i, name in enumerate(depth_names):
    depth_num = name.split()[-1]
    depth_descriptions[name] = f"Achieved a validation accuracy of {depth_accs[i]:.4f}, " + (
        f"indicating that {depth_num} provides sufficient capacity for waste classification."
        if i == depth_best_idx else
        f"showing {'limited capacity' if i < depth_best_idx else 'potential overfitting'} with {depth_num}.")

# GET REGULARIZATION DESCRIPTIONS
reg_descriptions = {}
for i, name in enumerate(reg_names):
    reg_type = name.split()[-1]
    if reg_type == "None":
        reg_descriptions[name] = f"Achieved a validation accuracy of {reg_accs[i]:.4f}, " + (
            "suggesting the model wasn't severely overfitting without regularization."
            if i == reg_best_idx else "but some form of regularization proved beneficial.")
    elif reg_type == "Dropout":
        reg_descriptions[name] = f"Achieved a validation accuracy of {reg_accs[i]:.4f}, " + (
            "effectively preventing overfitting by randomly deactivating neurons during training."
            if i == reg_best_idx else "possibly removing too many important features during training.")
    elif reg_type == "L2":
        reg_descriptions[name] = f"Achieved a validation accuracy of {reg_accs[i]:.4f}, " + (
            "effectively constraining weights to prevent overfitting."
            if i == reg_best_idx else "potentially constraining the model too much, resulting in lower accuracy.")
    elif reg_type == "BatchNorm":
        reg_descriptions[name] = f"Achieved a validation accuracy of {reg_accs[i]:.4f}, " + (
            "improving training stability and model performance through normalization."
            if i == reg_best_idx else "helping with training stability but not improving final accuracy sufficiently.")

# CREATING A DETAILED IMPACT ANALYSIS MARKDOWN DYNAMICALLY
impact_analysis = f"""
## Detailed Impact Analysis (Research Question 3b)

### Impact of Architecture Type
- **{arch_names[0]}**: {arch_descriptions[arch_names[0]]}
- **{arch_names[1]}**: {arch_descriptions[arch_names[1]]}
- **{arch_names[2]}**: {arch_descriptions[arch_names[2]]}
- **{arch_names[3]}**: {arch_descriptions[arch_names[3]]}

The architecture type impacts accuracy by affecting the model's ability to extract relevant features from waste images. {arch_best_name} performed best with an accuracy of {arch_best_acc:.4f}, suggesting that {'simpler models may be more effective at generalizing' if 'Simple' in arch_best_name else 'more complex architectures with specialized components provide advantages'} for this waste classification task.

### Impact of Filter Size
- **{filter_names[0]}**: {filter_descriptions[filter_names[0]]}
- **{filter_names[1]}**: {filter_descriptions[filter_names[1]]}
- **{filter_names[2]}**: {filter_descriptions[filter_names[2]]}

Filter size affects the model's ability to capture features at different scales. {filter_best_name} performed best with an accuracy of {filter_best_acc:.4f} because it provides {'the right balance between capturing local patterns while maintaining spatial resolution' if '3x3' in filter_best_name else 'a larger receptive field that captures more context' if '7x7' in filter_best_name else 'a good balance between detail and context'}, which is crucial for distinguishing between similar waste types.

### Impact of Network Depth
{chr(10).join([f"- **{name}**: {depth_descriptions[name]}" for name in depth_names])}

Network depth influences the model's capacity to learn hierarchical features. {depth_best_name} provided the optimal balance between feature extraction capability and model complexity, achieving an accuracy of {depth_best_acc:.4f}. The {'consistent improvement with increasing depth' if depth_best_idx == len(depth_names)-1 else 'peak at intermediate depth'} suggests that the waste classification task {'benefits from deeper feature hierarchies' if depth_best_idx > len(depth_names)/2 else 'does not require extremely deep networks'}.

### Impact of Regularization
{chr(10).join([f"- **{name}**: {reg_descriptions[name]}" for name in reg_names])}

Regularization techniques impact the model's ability to generalize. {reg_best_name} was most effective with an accuracy of {reg_best_acc:.4f}, {'suggesting that the dataset size and complexity were well-matched to the model capacity' if 'None' in reg_best_name else 'demonstrating the importance of preventing overfitting'}. This indicates that the waste classification task {'may benefit more from expressive models than from heavily regularized ones' if 'None' in reg_best_name else 'requires careful regularization to achieve optimal performance'}.

### Overall Impact Assessment
Based on our experiments, we can conclude that {overall_best[2]} has the greatest impact on waste classification accuracy, followed by {second_best[2]}. The {overall_best[0]} showed a clear advantage with a {overall_best[1]:.4f} validation accuracy.

The optimal configuration combines a {arch_best_name.split()[0]} architecture with {filter_best_name.split()[-1]} filters, {depth_best_name.split()[-1]} layers, and {reg_best_name.split()[-1]} regularization to achieve the highest accuracy while maintaining good generalization. This suggests that for waste classification, the ability to {'extract hierarchical features through sufficient network depth' if overall_best[2] == 'Network Depth' else 'use the right architectural pattern' if overall_best[2] == 'Architecture' else 'capture features at the appropriate scale' if overall_best[2] == 'Filter Size' else 'balance model capacity and generalization'} is most important for achieving high classification accuracy.
"""

display(Markdown(impact_analysis))